In [61]:
import numpy as np
import pickle
import scipy.io
import os

In [65]:
MIN_FRAMES = 500
PKT_LEN = 400
DIR_SOURCE = '/Users/stepanmazokha/Desktop/wisig_frames_rffi_dataset/node1-1/equalized_packets_min1000frames/'
FILE_TARGET = '/Users/stepanmazokha/Desktop/wisig_frames_rffi_dataset/node1-1/node1-1.pkl'

In [67]:
def process_save_rx(dir_source, file_target, frame_threshold, pkt_len):
    data_non_eq = []
    data_eq = []
    node_list = [] 

    # Work trough each TX file with frames
    for fname in os.listdir(dir_source):
        f = scipy.io.loadmat(dir_source + fname, verify_compressed_data_integrity=False)
        
        # Retrieve the list of frames; each item is a cell, containing two vectors: non-eq & eq IQ samples
        frames = f['packet_log'][0]
        node_name = fname[8:-4]

        # Don't process the file if not enough frames inside
        if len(frames) < frame_threshold:
            print(node_name,'Eliminated')
            continue
        # else: print(node_name, 'Processing')
        
        data_i_non_eq = np.zeros((frames.size, pkt_len, 2), dtype='float32')
        data_i_eq = np.zeros((frames.size, pkt_len, 2), dtype='float32')

        for frame_idx in np.arange(len(frames)):
            # Separately save real & iamginary parts of IQ samples for both non-equalized & equalized versions
            data_i_non_eq[frame_idx, :, 0] = np.real(frames[0][:, 0])
            data_i_non_eq[frame_idx, :, 1] = np.imag(frames[0][:, 0])

            data_i_eq[frame_idx, :, 0] = np.real(frames[0][:, 1])
            data_i_eq[frame_idx, :, 1] = np.imag(frames[0][:, 1])
                
        data_non_eq.append(data_i_non_eq)
        data_eq.append(data_i_eq)
        node_list.append(node_name)

    with open(file_target,'wb') as f:
        pickle.dump({
            'data_non_eq': data_non_eq,
            'data_eq': data_eq,
            'node_list': node_list
        },f)

    return [data_non_eq, data_eq, node_list]

data_non_eq, data_eq, node_list = process_save_rx(DIR_SOURCE, FILE_TARGET, MIN_FRAMES, PKT_LEN)